In [1]:
import pandas as pd
import glob
import numpy as np
import json
import sqlite3
import requests

# Carregar os dados de VRA
  - Normalizar o cabeçalho para snake case
  - Salvar estes dados

In [2]:
#função para salvar os dataframes em parquet
def saveParquet(df, nome):
    try:
        df.to_parquet(f"{nome}.parquet")
        print(f"Parquet tabela '{nome}' gerado com sucesso")
    except:
        print(f"Houve um erro na criação da tabela '{nome}'")

In [3]:
def getAllTableJson(arquivos, df):
    #for que concatena todos os arquivos csv gerando um unico dataframe
    for i in range(1, len(arquivos)):
        fileObject = open(arquivos[i], "r", encoding='utf-8-sig')
        jsonContent = fileObject.read()
        aList = json.loads(jsonContent)
        #dataframe
        df_get = pd.DataFrame(aList)

        #concatena os dataframes
        df = pd.concat([df, df_get], ignore_index=True)

    return df

In [4]:
def getVra():
    #define a pasta onde vai listar os arquivos
    folder_path = "VRA"
    #lista todos os arquivos para poder ler de uma vez
    arquivos = glob.glob(folder_path + "\*.json")

    #lista o primeiro dataframe para poder depois concatenar com os proximos
    fileObject = open(arquivos[0], "r", encoding='utf-8-sig')
    jsonContent = fileObject.read()
    aList = json.loads(jsonContent)
    #primeiro dataframe
    df = pd.DataFrame(aList)

    df_all = getAllTableJson(arquivos, df)

    df_all.columns = ['ICAO_EEMPRESA_AEREA', 'NUMERO_VOO', 'CODIGO_AUTORIZACAO', 'CODIGO_TIPO_LINHA', 'ICAO_AERODROMO_ORIGEM', 'ICAO_AERODROMO_DESTINO', 'PARTIDA_PREVISTA', 'PARTIDA_REAL', 'CHEGADA_PREVISTA', 'CHEGADA_REAL', 'SITUACAO_VOO', 'CODIGO_JUSTIFICATIVA']

    return df_all

In [5]:
#gerando o dataframe com todos os json
df_vra = getVra()
#salva em parquet
saveParquet(df_vra, 'vra')

#FIM de carregar os dados de VRA e salvar

Parquet tabela 'vra' gerado com sucesso


# Carregar dos dados de AIR_CIA
  - Normalizar o cabeçalho para snake case
  - Separar a coluna 'ICAO IATA' em duas colunas, seu conteúdo está separado por espaço e pode não conter o código IATA, caso não contenha o código IATA, deixe o valor nulo.
  - Salvar estes dados

In [6]:
def getAllTableCSV(arquivos, df):
    #for que concatena todos os arquivos csv gerando um unico dataframe
    for i in range(1, len(arquivos)):
        data = pd.DataFrame(pd.read_csv(arquivos[i], sep=';', na_filter=False))
        df = pd.concat([df, data], ignore_index=True)

    return df

In [7]:
def AirCia():
    #define a pasta onde vai listar os arquivos
    folder_path = "AIR_CIA"
    #lista todos os arquivos para poder ler de uma vez
    arquivos = glob.glob(folder_path + "\*.csv")

    #cria o primeiro dataframe para poder concatenar com os outros
    df_ANAC = pd.DataFrame(pd.read_csv(arquivos[0], sep=';', na_filter=False)) 

    df = getAllTableCSV(arquivos, df_ANAC)

    #aplicando snake_case
    df.columns = ['RAZAO_SOCIAL', 'ICAO_IATA', 'CNPJ', 'ATIVIDADES_AEREAS', 'ENDEREÇO_SEDE', 'TELEFONE', 'E_MAIL', 'DECISAO_OPERACIONAL', 'DATA_DECISAO_OPERACIONAL', 'VALIDADE_OPERACIONAL']
    
    #realiza split da coluna ICAO_IATA gerando duas novas colunas
    df_split_column = df.join(df['ICAO_IATA'].str.split(' ', expand=True, ).rename(columns={0:'ICAO', 1:'IATA'}))
    
    #select para excluir coluna antiga
    df_select = df_split_column[['RAZAO_SOCIAL', 'ICAO', 'IATA', 'CNPJ', 'ATIVIDADES_AEREAS', 'ENDEREÇO_SEDE', 'TELEFONE', 'E_MAIL', 'DECISAO_OPERACIONAL', 'DATA_DECISAO_OPERACIONAL', 'VALIDADE_OPERACIONAL']]
    

    return  df_select

In [8]:
#gerando o dataframe com todos os CSV
df_airCia = AirCia()
#salva o dataframe em PARQUET 
saveParquet(df_airCia, 'AirCia')

Parquet tabela 'AirCia' gerado com sucesso


# Criar nova tabela aerodromos
  - Através da API [https://rapidapi.com/Active-api/api/airport-info/]() trazer os aeródramos através do código ICAO presente nos dados de VRA.
  - Salvar estes dados

In [67]:
#lista os ICAO de cada coluna para depois juntar e comparar
lista_origem = df_vra['ICAO_AERODROMO_ORIGEM'].unique()
lista_destino = df_vra['ICAO_AERODROMO_DESTINO'].unique()

In [10]:
#gera a soma das duas listas de aerodromos sem repetição de ICAO
lista_aerdomos = list(set(lista_origem) | set(lista_destino))

In [11]:
def getAirport(icao):
    url = "https://airport-info.p.rapidapi.com/airport"

    #icao variavel para pegar cada um listado pela tabela vra
    querystring = {"icao":f"{icao}"}

    headers = {
        "X-RapidAPI-Key": "0fe0b39a83mshf2ed98c3d26c117p19b5c9jsnf5149cbcabc0",
        "X-RapidAPI-Host": "airport-info.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    json_data = json.loads(response.text)
    
    dfr = pd.DataFrame(json_data, index=[0])

    return dfr

In [12]:
#função que lista cada ICAO para extrair da api e gerar o dataframe final
def aerodromos():
    df_aerodromos = getAirport(lista_aerdomos[0])

    for icao in range(1, len(lista_aerdomos)):
        df_icao = getAirport(lista_aerdomos[icao])
        df_aerodromos = pd.concat([df_aerodromos, df_icao], ignore_index=True)
    
    return df_aerodromos

In [13]:
#extrai os aerodromos e retorna dataframe
df_aereodromos = aerodromos()
#salva o dataframe em PARQUET
saveParquet(df_aereodromos, 'aerodromos')

Parquet tabela 'aerodromos' gerado com sucesso


# Criar as seguintes views (Priorize o uso de SQL para esta parte):
  - Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
    - Razão social da companhia aérea
    - Nome Aeroporto de Origem
    - ICAO do aeroporto de origem
    - Estado/UF do aeroporto de origem
    - Nome do Aeroporto de Destino
    - ICAO do Aeroporto de destino
    - Estado/UF do aeroporto de destino

Para essa etapa irei gerar as tabelas no SQLITE3 para poder utilizar SQL. 

Ao inves de utilizar o spark sql do Databricks. Adaptando apenas por conta de utilizar o jupyter. 

In [14]:
conn = sqlite3.connect("pythonsqlite.db")

In [15]:
#criando as tabelas para gerenciar com o sql
df_aereodromos.to_sql("aerodromos", conn, if_exists="replace")
df_vra.to_sql("vra", conn, if_exists="replace")
df_airCia.to_sql("aircia", conn, if_exists="replace")

# Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
- Razão social da companhia aérea
- Nome Aeroporto de Origem
- ICAO do aeroporto de origem
- Estado/UF do aeroporto de origem
- Nome do Aeroporto de Destino
- ICAO do Aeroporto de destino
- Estado/UF do aeroporto de destino

In [16]:
rotas = pd.read_sql_query("""
SELECT A1.RAZAO_SOCIAL,
       R1.ICAO_EEMPRESA_AEREA,
       R1.ICAO_AERODROMO_ORIGEM,
       A2_ORIGEM.name AS NOME_AERODROMO_ORIGEM,
       A2_ORIGEM.location AS LOCATION_AERODROMO_ORIGEM,
       R1.ICAO_AERODROMO_DESTINO,
       A2_DESTINO.name AS NOME_AERODROMO_DESTINO,
       A2_DESTINO.location AS LOCATION_AERODROMO_DESTINO,
       max(R2.QTD_ROTA_UTILIZADA)
FROM vra R1
LEFT JOIN
  (SELECT ICAO_EEMPRESA_AEREA,
          ICAO_AERODROMO_ORIGEM,
          ICAO_AERODROMO_DESTINO,
          COUNT(*) AS QTD_ROTA_UTILIZADA
   FROM vra
   GROUP BY ICAO_EEMPRESA_AEREA,
            ICAO_AERODROMO_ORIGEM,
            ICAO_AERODROMO_DESTINO) R2 ON R1.ICAO_EEMPRESA_AEREA = R2.ICAO_EEMPRESA_AEREA
INNER JOIN aircia A1 ON R1.ICAO_EEMPRESA_AEREA = A1.ICAO
INNER JOIN 
(SELECT icao, name, location from aerodromos) A2_ORIGEM ON R1.ICAO_AERODROMO_ORIGEM = A2_ORIGEM.icao
INNER JOIN 
(SELECT icao, name, location from aerodromos) A2_DESTINO ON R1.ICAO_AERODROMO_ORIGEM = A2_DESTINO.icao
GROUP BY R1.ICAO_EEMPRESA_AEREA

""", con=conn)

In [28]:
#selecionando as colunas que utiliziará
view_rotas_mais_usadas = rotas[['RAZAO_SOCIAL', 'ICAO_EEMPRESA_AEREA', 'ICAO_AERODROMO_ORIGEM',	'NOME_AERODROMO_ORIGEM', 'LOCATION_AERODROMO_ORIGEM', 'ICAO_AERODROMO_DESTINO',	'NOME_AERODROMO_DESTINO', 'LOCATION_AERODROMO_DESTINO']]

In [29]:
#view das rota mais usada por empresa aerea
view_rotas_mais_usadas

,RAZAO_SOCIAL,ICAO_EEMPRESA_AEREA,ICAO_AERODROMO_ORIGEM,NOME_AERODROMO_ORIGEM,LOCATION_AERODROMO_ORIGEM,ICAO_AERODROMO_DESTINO,NOME_AERODROMO_DESTINO,LOCATION_AERODROMO_DESTINO
0,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),ACN,SBPA,Salgado Filho International Airport,"Porto Alegre, Rio Grande do Sul, Brazil",SSER,Salgado Filho International Airport,"Porto Alegre, Rio Grande do Sul, Brazil"
1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,ASO,SBCT,Afonso Pena International Airport,"Curitiba, Paraná, Brazil",SBFL,Afonso Pena International Airport,"Curitiba, Paraná, Brazil"
2,AZUL LINHAS AÉREAS BRASILEIRAS S/A,AZU,SBKP,Viracopos/Campinas International Airport,"São Paulo / Campinas, São Paulo, Brazil",SBFL,Viracopos/Campinas International Airport,"São Paulo / Campinas, São Paulo, Brazil"
3,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,GLO,SBGR,São Paulo–Guarulhos International Airport,"São Paulo, Brazil",SBTE,São Paulo–Guarulhos International Airport,"São Paulo, Brazil"
4,ABSA - AEROLINHAS BRASILEIRAS S.A.,LTG,SBGR,São Paulo–Guarulhos International Airport,"São Paulo, Brazil",SBEG,São Paulo–Guarulhos International Airport,"São Paulo, Brazil"
5,MODERN TRANSPORTE AEREO DE CARGA S.A,MWM,SBBE,Val de Cans International Airport,"Belém, Pará, Brazil",SBEG,Val de Cans International Airport,"Belém, Pará, Brazil"
6,OMNI TÁXI AÉREO S.A.,OMI,SBNF,Navegantes–Ministro Victor Konder Internationa...,"Navegantes, Santa Catarina, Brazil",SBPK,Navegantes–Ministro Victor Konder Internationa...,"Navegantes, Santa Catarina, Brazil"
7,MAP TRANSPORTES AÉREOS LTDA.,PAM,SBEG,Eduardo Gomes International Airport,"Manaus, Amazonas, Brazil",SBIH,Eduardo Gomes International Airport,"Manaus, Amazonas, Brazil"
8,PASSAREDO TRANSPORTES AÉREOS S.A.,PTB,SBSV,Deputado Luís Eduardo Magalhães International ...,"Salvador, Bahia, Brazil",SBMO,Deputado Luís Eduardo Magalhães International ...,"Salvador, Bahia, Brazil"
9,SIDERAL LINHAS AÉREAS LTDA.,SID,SBGR,São Paulo–Guarulhos International Airport,"São Paulo, Brazil",SBGO,São Paulo–Guarulhos International Airport,"São Paulo, Brazil"


# Para cada aeroporto trazer a companhia aérea com maior atuação no ano com as seguintes informações:
- Nome do Aeroporto
- ICAO do Aeroporto
- Razão social da Companhia Aérea
- Quantidade de Rotas à partir daquele aeroporto
- Quantidade de Rotas com destino àquele aeroporto
- Quantidade total de pousos e decolagens naquele aeroporto

In [93]:
cia_atuacao = pd.read_sql_query("""
    SELECT 
    R1.ICAO_AERODROMO_ORIGEM AS ICAO_AERODROMO, 
    A2.name AS NOME_AEROPORTO, 
    A2_ORIGEM.QTD_ROTA_ORIGEM,
    A2_DESTINO.QTD_ROTA_DESTINO,
    (A2_ORIGEM.QTD_ROTA_ORIGEM + A2_DESTINO.QTD_ROTA_DESTINO) AS QTD_TOTAL,
    R1.ICAO_EEMPRESA_AEREA, 
    A1.RAZAO_SOCIAL, 
    R2.ANO,
    max(R2.QTD_EMP_UTILIZADA)
    FROM vra R1
    LEFT JOIN 
        (SELECT ICAO_AERODROMO_ORIGEM, 
                ICAO_EEMPRESA_AEREA, 
                (strftime('%Y', CHEGADA_REAL)) AS ANO,
                COUNT(1) AS QTD_EMP_UTILIZADA
            FROM vra
            GROUP BY ICAO_AERODROMO_ORIGEM, 
                     ICAO_EEMPRESA_AEREA) R2 ON R1.ICAO_AERODROMO_ORIGEM = R2.ICAO_AERODROMO_ORIGEM
    INNER JOIN aircia A1 ON R1.ICAO_EEMPRESA_AEREA = A1.ICAO
    INNER JOIN aerodromos A2 ON  ICAO_AERODROMO = A2.icao
    LEFT JOIN
        (SELECT ICAO_AERODROMO_ORIGEM, 
               COUNT(ICAO_AERODROMO_ORIGEM) AS QTD_ROTA_ORIGEM
            FROM vra
            GROUP BY ICAO_AERODROMO_ORIGEM) A2_ORIGEM ON R1.ICAO_AERODROMO_ORIGEM = A2_ORIGEM.ICAO_AERODROMO_ORIGEM
    LEFT JOIN
        (SELECT ICAO_AERODROMO_DESTINO, 
               COUNT(ICAO_AERODROMO_DESTINO) AS QTD_ROTA_DESTINO
            FROM vra
            GROUP BY ICAO_AERODROMO_DESTINO) A2_DESTINO ON R1.ICAO_AERODROMO_ORIGEM = A2_DESTINO.ICAO_AERODROMO_DESTINO
    GROUP BY ICAO_AERODROMO
""", con=conn)

In [94]:
#selecionando as colunas que utiliziará
view_cia_atuacao = cia_atuacao[['ICAO_AERODROMO', 'NOME_AEROPORTO', 'QTD_ROTA_ORIGEM', 'QTD_ROTA_DESTINO', 'QTD_TOTAL', 'ICAO_EEMPRESA_AEREA', 'RAZAO_SOCIAL', 'ANO']]

In [95]:
#view das rota mais usada por empresa aerea
view_cia_atuacao

,ICAO_AERODROMO,NOME_AEROPORTO,QTD_ROTA_ORIGEM,QTD_ROTA_DESTINO,QTD_TOTAL,ICAO_EEMPRESA_AEREA,RAZAO_SOCIAL,ANO
0,EBBR,Brussels Airport (Zaventem Airport),140,135,275,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,2021
1,EDDF,Frankfurt Airport,1078,1066,2144,TAM,TAM LINHAS AÉREAS S.A.,2021
2,EHAM,Amsterdam Airport Schiphol,1196,1149,2345,TAM,TAM LINHAS AÉREAS S.A.,2021
3,KBOS,Logan International Airport,10,9,19,TAM,TAM LINHAS AÉREAS S.A.,None
4,KFLL,Fort Lauderdale–Hollywood International Airport,319,312,631,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,None
...,...,...,...,...,...,...,...,...
161,SWTS,Tangará da Serra Airport,193,184,377,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),2021
162,TJSJ,Luis Muñoz Marín International Airport,12,14,26,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,2021
163,TNCA,Queen Beatrix International Airport,2,2,4,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,2021
164,VTSK,Pattani Airport,4,4,8,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),None


In [96]:
#SALVANDO AS VIEWS
saveParquet(view_rotas_mais_usadas, 'v_rotas_mais_usadas')
saveParquet(view_cia_atuacao, 'v_cia_atuacao')

Parquet tabela 'v_rotas_mais_usadas' gerado com sucesso
Parquet tabela 'v_cia_atuacao' gerado com sucesso
